In [338]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)

<module 'model_afolu' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py'>

In [342]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape    

(36, 1575)

In [300]:
model_afolu.model_attributes.get_attribute_table("Land Use").key_values

['croplands',
 'forests_mangroves',
 'forests_primary',
 'forests_secondary',
 'grasslands',
 'other',
 'settlements',
 'wetlands']

In [327]:
v_fg = np.sum(model_afolu.arr_land_use[:,[1,2,3,4]], axis = 1)
v_crop = model_afolu.arr_land_use[:,0]

avg_change_fg = (v_fg[-1]-v_fg[0])/len(v_fg)
avg_change_crop = (v_crop[-1]-v_crop[0])/len(v_crop)
annual_change = avg_change_fg*50+avg_change_crop*40
delta_n = annual_change/10 # change in n
emit_kt_co2e = -delta_n*0.0055*(11/7)*310/1000
emit_kt_co2e


27.465738129144576

In [346]:
importlib.reload(sf)
importlib.reload(ma)
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))

model_afolu = ma.AFOLU(sa.model_attributes);
df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_project_afolu[[x for x in df_project_afolu.columns if "fertilizer" in x] + [x for x in df_project_afolu.columns if ("pasture" in x) and ("qty_lsmm" in x)]]
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_ch4" in x)]].sum(axis = 1))
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_n2o" in x)]].sum(axis = 1))
# about 2 billion kg/
#qty_lsmm_n_to_pastures_
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("lvst" in x)]]
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("agrc" in x)]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("agrc" in x)]].head()


arr_soil_frac_cur_drywet_crop_synthetic for Agriculture Fraction Dry:
[23.8138397  24.69492295 25.1248551  25.59679536]
arr_soil_frac_cur_drywet_crop_synthetic for Agriculture Fraction Wet:
[30.32965135 30.09413448 30.61577418 31.19293762]
F_ON and F_SN
[0.16939933 0.1692346  0.17082108 0.1719929  0.17321667 0.174297
 0.17537093 0.1764267  0.17753054 0.17868692 0.17981006 0.18088768
 0.18191222 0.18288066 0.18378989 0.18463856 0.18542725 0.18615721
 0.1868256  0.18743381 0.18797957 0.18846266 0.18888177 0.18923537
 0.18952187 0.18973926 0.18988418 0.18995427 0.18994573 0.18985377
 0.18967332 0.19137924 0.19345469 0.19555156 0.19767038 0.19981014]
[0.54039257 0.54599727 0.55367509 0.5618655  0.57024898 0.57864242
 0.58733458 0.59640124 0.60603539 0.61591613 0.62555569 0.63495913
 0.64414035 0.65311632 0.66190676 0.67051089 0.67892435 0.68715612
 0.69521923 0.70312746 0.71089531 0.71853693 0.72606567 0.73349388
 0.74083288 0.74809269 0.75528128 0.76240427 0.76946441 0.77646071
 0.7833884

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:485: RuntimeWarning: divide by zero encountered in true_divide
  vec_lvst_cc_init = vec_lvst_pop_init/(vec_initial_area[ind_pstr]*vec_lvst_pstr_weights)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:522: RuntimeWarning: invalid value encountered in multiply
  vec_lvst_prod_proj = vec_lvst_cc_proj*area_pstr_proj*vec_lvst_pstr_weights
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1973: RuntimeWarning: invalid value encountered in true_divide
  arr_cur = np.nan_to_num(arr_cur/arr, 0.0)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1548: UserWarning: Missing fields 'area_agrc_crops_bevs_and_spices', 'area_agrc_crops_cereals', 'area_agrc_crops_fibers', 'area_agrc_crops_fruits', 'area_agrc_crops_herbs_and_other_perennial_crops', 'area_agrc_crops_nuts', 'area_agrc_crops_other_annual', 'area_agr

,emission_co2e_ch4_agrc_crop_activity_rice,emission_co2e_co2_agrc_crop_activity_bevs_and_spices,emission_co2e_co2_agrc_crop_activity_cereals,emission_co2e_co2_agrc_crop_activity_fibers,emission_co2e_co2_agrc_crop_activity_fruits,emission_co2e_co2_agrc_crop_activity_herbs_and_other_perennial_crops,emission_co2e_co2_agrc_crop_activity_nuts,emission_co2e_co2_agrc_crop_activity_other_annual,emission_co2e_co2_agrc_crop_activity_other_woody_perennial,emission_co2e_co2_agrc_crop_activity_pulses,...,emission_co2e_n2o_agrc_crop_activity_herbs_and_other_perennial_crops,emission_co2e_n2o_agrc_crop_activity_nuts,emission_co2e_n2o_agrc_crop_activity_other_annual,emission_co2e_n2o_agrc_crop_activity_other_woody_perennial,emission_co2e_n2o_agrc_crop_activity_pulses,emission_co2e_n2o_agrc_crop_activity_rice,emission_co2e_n2o_agrc_crop_activity_sugar_cane,emission_co2e_n2o_agrc_crop_activity_tubers,emission_co2e_n2o_agrc_crop_activity_vegetables_and_vines,emission_co2e_subsector_total_agrc
0,0.133150,0.012370,0.000600,0.000125,0.010492,0.000419,0.007389,0.000302,0.012787,0.000139,...,0.045746,0.111102,0.063246,0.111296,0.013582,0.0,0.044067,0.006614,0.009445,1.018843
1,0.134773,0.012572,0.000605,0.000126,0.010620,0.000426,0.007480,0.000306,0.012849,0.000141,...,0.046490,0.112456,0.063916,0.111830,0.013747,0.0,0.044369,0.006695,0.009560,1.030264
2,0.137190,0.012858,0.000613,0.000127,0.010811,0.000435,0.007614,0.000311,0.012957,0.000143,...,0.047548,0.114473,0.064935,0.112772,0.013994,0.0,0.044927,0.006815,0.009732,1.047565
3,0.139745,0.013149,0.000622,0.000129,0.011012,0.000445,0.007755,0.000316,0.013081,0.000146,...,0.048624,0.116605,0.066026,0.113853,0.014255,0.0,0.045618,0.006942,0.009913,1.066059
4,0.141133,0.013460,0.000633,0.000131,0.011227,0.000456,0.007907,0.000321,0.013211,0.000149,...,0.049775,0.118877,0.067187,0.114986,0.014532,0.0,0.046439,0.007077,0.010106,1.084666


In [168]:
importlib.reload(sf)
importlib.reload(ma)
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))

model_afolu = ma.AFOLU(sa.model_attributes);
df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_project_afolu[[x for x in df_project_afolu.columns if "fertilizer" in x] + [x for x in df_project_afolu.columns if ("pasture" in x) and ("qty_lsmm" in x)]]
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_ch4" in x)]].sum(axis = 1))
#print(df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("lsmm" in x) and ("co2e_n2o" in x)]].sum(axis = 1))
# about 2 billion kg/
#qty_lsmm_n_to_pastures_
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("lvst" in x)]]
df_project_afolu[[x for x in df_project_afolu.columns if ("emission_co2e" in x) and ("agrc" in x)]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu.columns if ("emission" in x) and ("agrc" in x)]].head()


[58.9        59.57744243 60.41909385 61.31681376 62.23594458 63.15617228
 64.10888921 65.10221154 66.1571338  67.23811735 68.29140823 69.31748291
 70.31776177 71.29398869 72.24818966 73.18009738 74.08906272 74.97592487
 75.84196491 76.68855218 77.51715115 78.32921305 79.12613925 79.90926598
 80.67987768 81.43918038 82.18823271 82.92791673 83.65888699 84.38147522
 85.09568936 85.90940701 86.69843408 87.48776414 88.28367223 89.09311415]
[40.5        40.96581355 41.54453822 42.16181591 42.79381588 43.42657007
 44.08166406 44.76467856 45.49004955 46.23334045 46.9575897  47.66312492
 48.35092278 49.02218238 49.67829679 50.31908224 50.94409236 51.5539042
 52.14939862 52.7315172  53.30126692 53.85964565 54.40761697 54.9460997
 55.47597702 55.99807819 56.51313115 57.0217424  57.52436202 58.0212181
 58.51231611 59.07183334 59.61437318 60.15712135 60.70439262 61.26096983]
[15.86608506 15.9484422  16.09791424 16.20777967 16.32187356 16.42179696
 16.52046081 16.61673261 16.71679176 16.82105458 16.

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:464: RuntimeWarning: divide by zero encountered in true_divide
  vec_lvst_cc_init = vec_lvst_pop_init/(vec_initial_area[ind_pstr]*vec_lvst_pstr_weights)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_afolu.py:501: RuntimeWarning: invalid value encountered in multiply
  vec_lvst_prod_proj = vec_lvst_cc_proj*area_pstr_proj*vec_lvst_pstr_weights
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1930: RuntimeWarning: invalid value encountered in true_divide
  arr_cur = np.nan_to_num(arr_cur/arr, 0.0)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:1505: UserWarning: Missing fields 'area_agrc_crops_bevs_and_spices', 'area_agrc_crops_cereals', 'area_agrc_crops_fibers', 'area_agrc_crops_fruits', 'area_agrc_crops_herbs_and_other_perennial_crops', 'area_agrc_crops_nuts', 'area_agrc_crops_other_annual', 'area_agr

,emission_co2e_ch4_agrc_crop_activity_rice,emission_co2e_co2_agrc_crop_activity_bevs_and_spices,emission_co2e_co2_agrc_crop_activity_cereals,emission_co2e_co2_agrc_crop_activity_fibers,emission_co2e_co2_agrc_crop_activity_fruits,emission_co2e_co2_agrc_crop_activity_herbs_and_other_perennial_crops,emission_co2e_co2_agrc_crop_activity_nuts,emission_co2e_co2_agrc_crop_activity_other_annual,emission_co2e_co2_agrc_crop_activity_other_woody_perennial,emission_co2e_co2_agrc_crop_activity_pulses,...,emission_co2e_n2o_agrc_crop_activity_herbs_and_other_perennial_crops,emission_co2e_n2o_agrc_crop_activity_nuts,emission_co2e_n2o_agrc_crop_activity_other_annual,emission_co2e_n2o_agrc_crop_activity_other_woody_perennial,emission_co2e_n2o_agrc_crop_activity_pulses,emission_co2e_n2o_agrc_crop_activity_rice,emission_co2e_n2o_agrc_crop_activity_sugar_cane,emission_co2e_n2o_agrc_crop_activity_tubers,emission_co2e_n2o_agrc_crop_activity_vegetables_and_vines,emission_co2e_subsector_total_agrc
0,0.133150,0.012370,0.000600,0.000125,0.010492,0.000419,0.007389,0.000302,0.012787,0.000139,...,0.045746,0.111102,0.063246,0.111296,0.013582,0.0,0.044067,0.006614,0.009445,1.018843
1,0.134773,0.012572,0.000605,0.000126,0.010620,0.000426,0.007480,0.000306,0.012849,0.000141,...,0.046490,0.112456,0.063916,0.111830,0.013747,0.0,0.044369,0.006695,0.009560,1.030264
2,0.137190,0.012858,0.000613,0.000127,0.010811,0.000435,0.007614,0.000311,0.012957,0.000143,...,0.047548,0.114473,0.064935,0.112772,0.013994,0.0,0.044927,0.006815,0.009732,1.047565
3,0.139745,0.013149,0.000622,0.000129,0.011012,0.000445,0.007755,0.000316,0.013081,0.000146,...,0.048624,0.116605,0.066026,0.113853,0.014255,0.0,0.045618,0.006942,0.009913,1.066059
4,0.141133,0.013460,0.000633,0.000131,0.011227,0.000456,0.007907,0.000321,0.013211,0.000149,...,0.049775,0.118877,0.067187,0.114986,0.014532,0.0,0.046439,0.007077,0.010106,1.084666


In [136]:
df_project_afolu[[x for x in df_project_afolu.columns if ("qty" in x) and ("lsmm" in x)]].head()
#df_project_afolu[[x for x in df_project_afolu.columns if ("qty_lsmm_n_to_fertilizer_" in x) and ("from" not in x)]].sum(axis = 1).head()

,qty_lsmm_dung_to_incinerate_tonne,qty_lsmm_total_n_to_pastures_kt,qty_lsmm_n_to_fertilizer_anaerobic_digester_kt,qty_lsmm_n_to_fertilizer_anaerobic_lagoon_kt,qty_lsmm_n_to_fertilizer_composting_kt,qty_lsmm_n_to_fertilizer_daily_spread_kt,qty_lsmm_n_to_fertilizer_deep_bedding_kt,qty_lsmm_n_to_fertilizer_dry_lot_kt,qty_lsmm_n_to_fertilizer_incineration_kt,qty_lsmm_n_to_fertilizer_liquid_slurry_kt,qty_lsmm_n_to_fertilizer_paddock_pasture_range_kt,qty_lsmm_n_to_fertilizer_poultry_manure_kt,qty_lsmm_n_to_fertilizer_storage_solid_kt,qty_lsmm_total_n_used_for_other_kt,qty_lsmm_total_n_to_fertilizer_from_dung_kt,qty_lsmm_total_n_to_fertilizer_from_urine_kt
0,0.0,69.598279,0.0,0.009229,0.0,4.870467,0.0,7.397749,0.0,0.058362,0.0,0.045818,3.484461,1.762898,7.933043,7.933043
1,0.0,69.700295,0.0,0.009229,0.0,4.972655,0.0,7.366903,0.0,0.058365,0.0,0.045838,3.495453,1.772049,7.974221,7.974221
2,0.0,70.318122,0.0,0.009314,0.0,5.048185,0.0,7.411581,0.0,0.058909,0.0,0.046278,3.523648,1.788657,8.048957,8.048957
3,0.0,70.794746,0.0,0.009381,0.0,5.098707,0.0,7.449134,0.0,0.059331,0.0,0.046625,3.544603,1.800864,8.103890,8.103890
4,0.0,71.274163,0.0,0.009447,0.0,5.156687,0.0,7.483360,0.0,0.059753,0.0,0.046974,3.565652,1.813542,8.160937,8.160937


In [148]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape                      
#self.model_attributes.get_standard_variables(df_afolu_trajectories, self.modvar_lvst_animal_weight, True, "array_base", expand_to_all_cats = True)                        

(36, 1514)

In [341]:
#vars_afolu = sa.model_attributes.build_varlist("Soil Management", variable_type = "input")
vars_afolu = sa.model_attributes.build_varlist("Land Use", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Land Use", variable_type = "input")
vars_afolu = [x for x in vars_afolu if (x not in df_afolu_trajectories.columns)]
vars_afolu.sort()

for k in vars_afolu:
    print(k)
    

factor_lndu_soil_carbon_croplands
factor_lndu_soil_carbon_forests_mangroves
factor_lndu_soil_carbon_forests_primary
factor_lndu_soil_carbon_forests_secondary
factor_lndu_soil_carbon_grasslands
factor_lndu_soil_carbon_other
factor_lndu_soil_carbon_settlements
factor_lndu_soil_carbon_wetlands


In [409]:
print(model_afolu.modvar_lsmm_emissions_n2o)

:math:\text{N}_2\text{O} Emissions from Manure Management


In [80]:

#df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")





In [16]:
dict_arrs_lsmm_frac_manure = sa.model_attributes.get_multivariables_with_bounded_sum_by_category(
    df_afolu_trajectories,
    model_afolu.modvar_lvst_list_mm_fractions,
    1,
    force_sum_equality = True,
    msg_append = "Energy fractions by category do not sum to 1. See definition of dict_arrs_inen_frac_energy."
)

In [26]:
sa.model_attributes.get_standard_variables(df_afolu_trajectories, model_afolu.modvar_lvst_genfactor_volatile_solids)

,genfactor_lvst_daily_volatile_solids_buffalo,genfactor_lvst_daily_volatile_solids_cattle_dairy,genfactor_lvst_daily_volatile_solids_cattle_nondairy,genfactor_lvst_daily_volatile_solids_chickens,genfactor_lvst_daily_volatile_solids_goats,genfactor_lvst_daily_volatile_solids_horses,genfactor_lvst_daily_volatile_solids_mules,genfactor_lvst_daily_volatile_solids_pigs,genfactor_lvst_daily_volatile_solids_sheep
0,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
1,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
2,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
3,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
4,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
5,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
6,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
7,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
8,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
9,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
